This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [3]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missin this module, simply run `pip install python-dotenv`

load_dotenv()
pierreKey = os.getenv('NASDAQ_API_KEY')

#print(pierreKey)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# First, import the relevant modules
import json
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [7]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
re='https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2020-12-01&end_date=2020-12-03&api_key='
req=re+pierreKey
#print(re)
r=requests.get(req)
print(r.text)

{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2020-12-01","end_date":"2020-12-01","frequency":"daily","data":[["2020-12-01",112.2,112.2,111.5,112.0,null,51.0,5703.0,null,null,null]],"collapse":null,"order":null}}


In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [9]:
req='https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=JmmAsy5eSmvidqxBF9Re'
t=requests.get(req)
# for testing only t='''{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2020-12-01","end_date":"2020-12-01","frequency":"daily","data":[["2020-12-01",112.2,112.2,111.5,112.0,null,53.0,5703.0,null,null,null],["2020-12-02",110.0,114.2,109.5,112.2,null,51.0,5703.0,null,null,null]],"collapse":null,"order":null}}'''
o=json.loads(t.text) #main dictionary object
d=o['dataset_data']['data'] #daily list information
#print(json.dumps(d))

#NOTE, we perform separate loops for each question below.
#We could optimize the code to only parse the data once as a future exercice

#3- let's compute the lowest and highest opening prices for the time range.
min=-1
max=0
for i in d:
    if i[1]: #test for null values
        if min == -1:
                min = i[1] #opening price is the second element
                max = i[1]
        else:
            if i[1] < min: min=i[1]
            if i[1] > max: max=i[1]
print("For request =",re,"we got",len(d),"data points")
print("Minimum opening price is:",str(min),", max opening price is:",str(max))

#4-computing max difference in one day, high is index 2, low is 3
diff=-1
for i in d:
    if i[2] and i[3]:
        if diff == -1: 
            diff = i[2]-i[3]
        else:
            if i[2]-i[3] > diff: diff= i[2]-i[3]
print("Maximum price difference in a day is:",str(round(diff,2)))

#5-computing max difference between 2 consecutive days, based on closing price
diff=0
for i in range(0,len(d)-2):
    if d[i+1][4] and d[i][4]:
        if diff < d[i+1][4]-d[i][4]: 
            diff = d[i+1][4]-d[i][4]
print("Maximum price difference between 2 consecutive days is:",str(round(diff,2)))

#6-Compute the average daily trading volume during this year - index 6 in data
sum=0
tot=0
for i in d:
    if i[6]:
        sum=sum+i[6]
        tot=tot+1
print("Average daily trading volume is:",str(round(sum/tot,2)))

#7-Compute the median trading volume during this year
def median(l):
    n = len(l)
    s = sorted(l)
    return (s[n//2-1]/2.0+s[n//2]/2.0, s[n//2])[n % 2] if n else None

vol=[]
for i in d:
    if i[6]:
        vol.append(i[6])
print("Median daily trading volume is:",str(round(median(vol),2)))

For request = https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2020-12-01&end_date=2020-12-03&api_key= we got 255 data points
Minimum opening price is: 34.0 , max opening price is: 53.11
Maximum price difference in a day is: 2.81
Maximum price difference between 2 consecutive days is: 2.56
Average daily trading volume is: 89124.34
Median daily trading volume is: 76286.0
